# Avaliacão Módulo II

Faca um programa que:
1) Carregue os dados de um csv remoto, ex. csv do github, csv de uma base aberta qualquer ou API.  
2) Crie uma api que filtre esses dados e retorne o json dos dados filtrados.  
3) A partir dos dados filtrados você deve escolher uma variável quantitativa para agregar com um groupby e criar um csv e um json resposta, ambos devem ser persistidos.  
4) A partir dos dados filtrados você deve escolher um gráfico para fazer como por exemplo fizemos as gorgetas dadas por sexo ou dia da semana. Crie uma figura png ou jpeg resposta.  
5) Coloque sua resposta em um projeto novo no git e envie por e-mail.  
  
Entregar até 29/11.

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('http://dados.recife.pe.gov.br/dataset/4d3a3b39-9ea9-46ed-bf21-a2670de519c1/resource/7c613836-9edd-4c0f-bc72-495008dd29c3/download/info_unidadesensino_07102021.csv',sep=';', skipinitialspace = True)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351 entries, 0 to 350
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   rpa                 351 non-null    int64  
 1   tipo_cod            351 non-null    int64  
 2   tipo                351 non-null    object 
 3   cod_escola          351 non-null    int64  
 4   escola              351 non-null    object 
 5   inep                349 non-null    float64
 6   rua                 345 non-null    object 
 7   numero              351 non-null    object 
 8   cod_bairro          351 non-null    int64  
 9   bairro              351 non-null    object 
 10  metragem            312 non-null    object 
 11  qtd_alunos          351 non-null    int64  
 12  qtd_turmas          351 non-null    int64  
 13  qtd_professores     351 non-null    int64  
 14  escola_climatizada  351 non-null    object 
 15  data_visita         125 non-null    object 
 16  quadra_c

In [4]:
data['bairro'].unique()

array(['CABANGA', 'SANTO AMARO', 'ILHA JOANA BEZERRA', 'COELHOS',
       'RECIFE', 'BOA VISTA', 'ILHA DO LEITE', 'SAO JOSE', 'AGUA FRIA',
       'DOIS UNIDOS', 'FUNDAO', 'CAMPINA DO BARRETO',
       'ALTO SANTA TEREZINHA', 'CAJUEIRO', 'TORREAO',
       'BOMBA DO HEMETERIO', 'BEBERIBE', 'ENCRUZILHADA',
       'ALTO JOSE BONIFACIO', 'CAMPO GRANDE', 'PEIXINHOS',
       'LINHA DO TIRO', 'ARRUDA', 'BREJO DA GUABIRABA',
       'BREJO DE BEBERIBE', 'NOVA DESCOBERTA', 'MACAXEIRA',
       'CORREGO DO JENIPAPO', 'VASCO DA GAMA', 'GUABIRABA', 'PASSARINHO',
       'MORRO DA CONCEICAO', 'MANGABEIRA', 'TAMARINEIRA', 'ESPINHEIRO',
       'CASA AMARELA', 'ALTO DO MANDU', 'SITIO DOS PINTOS', 'DERBY',
       'MONTEIRO', 'ALTO JOSE DO PINHO', 'DOIS IRMAOS', 'PARNAMIRIM',
       'SANTANA', 'APIPUCOS', 'TORROES', 'IPUTINGA', 'VARZEA', 'CORDEIRO',
       'PRADO', 'ENGENHO DO MEIO', 'ILHA DO RETIRO', 'CAXANGA', 'TORRE',
       'MADALENA', 'CIDADE UNIVERSITARIA', 'COQUEIRAL', 'ESTANCIA',
       'SAN MARTIN', 

In [5]:
data.head()

,rpa,tipo_cod,tipo,cod_escola,escola,inep,rua,numero,cod_bairro,bairro,...,qtd_professores,escola_climatizada,data_visita,quadra_coberta,quadra_descoberta,biblioteca,sala_recurso,gestor,longitude,latitude
0,1,1,ESCOLA MUNICIPAL,90,ALMIRANTE SOARES DUTRA,26120658.0,CAMUTANGA,180,51,CABANGA,...,12,SIM,27/2/2018,NÃO,NÃO,SIM,SIM,VERONICA MARIA LIMA,-34.89535,-8.07895
1,1,1,ESCOLA MUNICIPAL,289,CIDADAO HERBERT DE SOUZA,26121972.0,ARNOBIO MARQUES,310,108,SANTO AMARO,...,12,NÂO,NaN,NÃO,NÃO,NÃO,SIM,ANA CLAUDIA DO NASCIMENTO SILVA,-34.88707,-8.04576
2,1,1,ESCOLA MUNICIPAL,256,DO COQUE,26122898.0,MIRANDOPOLIS,35,43,ILHA JOANA BEZERRA,...,22,NÂO,NaN,NÃO,NÃO,SIM,SIM,GRIJALBA MARIA PESSOA,-34.90022,-8.07145
3,1,1,ESCOLA MUNICIPAL,91,DOS COELHOS,26118670.0,DOS COELHOS,591,60,COELHOS,...,28,NÂO,NaN,NÃO,NÃO,SIM,SIM,FRANCISCA ANGELA DA SILVA,-34.89097,-8.06876
4,1,1,ESCOLA MUNICIPAL,384,FREI TADEU GLASER,26175347.0,AGAMENON MAGALHAES,600,108,SANTO AMARO,...,7,NÂO,19/4/2017,NÃO,NÃO,SIM,NÃO,KIRIA CELESTE DORNELAS DA SILVA,-34.87818,-8.04006


In [6]:
data.to_csv('escolas.csv', index=False)

In [7]:
bairro='afogados'
bairro_procurado = (data['bairro'] == bairro.upper())
escolas_bairro = data[bairro_procurado]

In [8]:
data.set_index('cod_escola', inplace=True)

In [9]:
data['tipo_cod'].unique()

array([1, 2, 5, 8], dtype=int64)

In [10]:
%%writefile api.py
from flask import Flask, make_response
from flask_restful import Resource, Api
import pandas as pd

app = Flask(__name__)
api = Api(app)

df = pd.read_csv('escolas.csv')
df.set_index('cod_escola')

class Bairro(Resource):
    def get(self, bairro):
        bairro_procurado = (df['bairro'] == bairro.upper())
        escolas_bairro = df[bairro_procurado]
        escolas_bairro.to_csv(f'escolas_{bairro}.csv')
        return make_response(escolas_bairro.to_json(orient='index',  force_ascii=False))

class Escola(Resource):
    def get(self,cod_escola):
        escola_procurada = df.loc[[cod_escola]]
        return make_response(escola_procurada.to_json(orient='index',force_ascii=False))

class Tipo(Resource):
    def get(self, tipo_cod):
        tipo_procurado = (df['tipo_cod'] == tipo_cod)
        escolas_tipo = df[tipo_procurado]
        escolas_tipo.to_csv(f'tipo_{tipo_cod}.csv')
        return make_response(escolas_tipo.to_json(orient='index',  force_ascii=False))
    
api.add_resource(Bairro, '/bairro/<string:bairro>')
api.add_resource(Escola, '/escola/<int:cod_escola>')
api.add_resource(Tipo, '/tipo/<int:tipo_cod>')

if __name__ == '__main__':
    app.run(debug=True)

Writing api.py


In [37]:
!python api.py

^C
